In [1]:
%matplotlib widget
import json

import numpy as np
import matplotlib.pyplot as plt
plt.ioff()

from matplotlib.widgets import Slider
import ipywidgets as ipw
from functools import partial

In [2]:
# Define the main folder and parameters
DATA_DIR = "./data"
FORCE_ON_DISP_ATOM_KEY = "force_on_disp_atom"
ENERGY_PER_ATOM_KEY = "energy_per_atom"
SMEARING_KEY = "smearing"
KPOINT_DISTANCE_KEY = "kpoint_distance"

unit_dict = {
    ENERGY_PER_ATOM_KEY: "eV/atom",
    FORCE_ON_DISP_ATOM_KEY: "eV/Å",
}
label_dict = {
    ENERGY_PER_ATOM_KEY: "energy per atom",
    FORCE_ON_DISP_ATOM_KEY: "force on displacement atom",
}

# SSSP protocols convergence study

This page shows interactive plots of the convergence of the magnitude of the force on an single displaced atom (top panel), and of the total energy per atom (bottom panel) as a function of $k$-point sampling (encoded by the $k$-points distance $\lambda$) and smearing temperature $\sigma$. 

Different error thresholds are considered to evaluate convergence. Systematic errors are governed by smearing temperature, while $k$-point errors arise from undersampling of the Brillouin zone (with coarser $k$-point meshes). "Convergence", in this case, is considered reached by calculations that produce both systematic and sampling errors below their respective thresholds, as set by the sliders. Heatmap colors, as well as the numberical values in each cell, encode the percentage of materials that are converged within the respective thresholds, for the given calculation parameters.

The data behind this plot encompass 124 2D materials (encompassing metals and insulators) and 145 3D metals, randomly sampled from the [MC2D](https://mc2d.materialscloud.org) and [MC3D](https://mc3d.materialscloud.org) databases, respectively. DFT calculations were performed with the [Quantum ESPRESSO package](https://www.quantum-espresso.org), using the Marzari-Vanderbilt-DeVita-Payne cold smearing scheme (see [PRL 82, 3296 (1999)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.82.3296)) and automated with [AiiDA](https://www.aiida.net).

For further details on the benchmark workflows and analysis of errors, please refer to the [SSSP-protocols paper]().

In [3]:
def create_figure():
    coefficient = 0.8
    figsize = (13 * coefficient, 7 * coefficient)

    fig = plt.figure(figsize=figsize)
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False

    return fig


In [4]:
# Define the main functions
def place_annotations(data, ax, kpoint_distances, smearing_values):
    for i in range(len(kpoint_distances)):
        for j in range(len(smearing_values)):
            value = data['values'][i, j]
            color = "white" if value > 0.5 else "0.5"
            _ = ax.text(
                j,
                i,
                f"{value:.1%}",
                ha="center",
                va="center",
                color=color,
                fontsize=9,
            )

def write_texts(ax, smearing_values, property_key):
    kpt_meshes = [
        "6x6x1",
        "7x7x1",
        "9x9x1",
        "12x12x1",
        "17x17x1",
        "35x35x1",
        "50x50x1",
    ]
    ys = ax.get_yticks()
    x = len(smearing_values) + 0.2
    for i in range(len(ys)):
        ax.text(
            x,
            ys[i],
            f"{kpt_meshes[i]}",
            ha="center",
            va="center",
            color="black",
            fontsize=10,
        )
    ax.text(
        x,
        min(ys) - 0.7,
        "Average\n$k$-point\nmesh size:",
        ha="center",
        va="center",
        color="black",
        fontsize=8,
    )
    ax.set_title(f"Convergence of {label_dict[property_key]}", fontsize=14)

def erase_texts(obj):
    for txt in obj.texts:
        txt.set_visible(False)

def update_on_slider_change(event, fig, ax, im, all_data, syst_thr_slider, kpts_thr_slider, available_systematic_thresholds, available_kpt_thresholds, kpoint_distances, smearing_values, property_key):
        data = get_closest_df(
            all_data,
            syst_thr_slider.val,
            kpts_thr_slider.val,
            available_systematic_thresholds,
            available_kpt_thresholds,
        )
        im.set_data(data['values'])
        erase_texts(ax)
        place_annotations(data, ax, kpoint_distances, smearing_values)
        write_texts(ax, smearing_values, property_key)
        fig.canvas.draw_idle()

def place_sliders(fig, property_key, slider_lims, default_systematic_error_thr, default_kpts_error_thr):
    fig.subplots_adjust(left=0.2, bottom=0.2)
    ax_hslider = fig.add_axes([0.25, 0.1, 0.63, 0.03])
    syst_thr_slider = Slider(
        ax=ax_hslider,
        label=f"Threshold on systematic\nerror from smearing ({unit_dict[property_key]})",
        valmin=slider_lims[property_key]["syst_thr"][0],
        valmax=slider_lims[property_key]["syst_thr"][1],
        valinit=default_systematic_error_thr[property_key],
    )
    ax_vslider = fig.add_axes([0.1, 0.25, 0.0225, 0.63])
    kpts_thr_slider = Slider(
        ax=ax_vslider,
        label=f"Threshold on\nk-point convergence\n({unit_dict[property_key]})",
        valmin=slider_lims[property_key]["kpts_thr"][0],
        valmax=slider_lims[property_key]["kpts_thr"][1],
        valinit=default_kpts_error_thr[property_key],
        orientation="vertical",
    )
    syst_thr_slider.label.set_size(12)
    kpts_thr_slider.label.set_size(12)
    return syst_thr_slider, kpts_thr_slider

def get_closest_df(
    data_dict,
    systematic_threshold,
    kpt_threshold,
    available_systematic_thresholds,
    available_kpt_thresholds,
):
    systematic_threshold_idx = np.argmin(
        np.abs(available_systematic_thresholds - systematic_threshold)
    )
    kpt_threshold_idx = np.argmin(np.abs(available_kpt_thresholds - kpt_threshold))
    systematic_thr, kpt_thr = (
        available_systematic_thresholds[systematic_threshold_idx],
        available_kpt_thresholds[kpt_threshold_idx],
    )
    return data_dict[systematic_thr, kpt_thr]


def get_available_params(data_dict):
    available_params = list(data_dict.keys())
    available_systematic_thresholds = np.unique(
        np.array([p[0] for p in available_params])
    )
    available_kpt_thresholds = np.unique(np.array([p[1] for p in available_params]))
    return available_systematic_thresholds, available_kpt_thresholds


def plot_convergence_heatmap(property_key, fig):
    with open(f"{DATA_DIR}/{property_key}.json", "rb") as f:
        raw_data = json.load(f)
        all_data = {
            tuple(datapoint['coords']): 
                {
                    'values': np.array(datapoint['values']),
                    'columns': np.array(datapoint['columns']),
                    'index': np.array(datapoint['index']),
                }
            for datapoint in raw_data
        }

    available_systematic_thresholds, available_kpt_thresholds = get_available_params(
        all_data
    )
    default_systematic_error_thr = {
        ENERGY_PER_ATOM_KEY: 0.02,
        FORCE_ON_DISP_ATOM_KEY: 0.035,
    }
    default_kpts_error_thr = {
        ENERGY_PER_ATOM_KEY: 0.002,
        FORCE_ON_DISP_ATOM_KEY: 0.0035,
    }

    slider_lims = {
        ENERGY_PER_ATOM_KEY: {
            "kpts_thr": (0.001, 0.03),
            "syst_thr": (0.001, 0.03),
        },
        FORCE_ON_DISP_ATOM_KEY: {
            "kpts_thr": (0.002, 0.01),
            "syst_thr": (0.002, 0.1),
        },
    }

    data = get_closest_df(
        all_data,
        default_systematic_error_thr[property_key],
        default_kpts_error_thr[property_key],
        available_systematic_thresholds,
        available_kpt_thresholds,
    )

    smearing_values = sorted(list(data['columns']))
    kpoint_distances = sorted(list(data['index']))

    ax = fig.axes[0] if fig.axes else fig.add_subplot(111)
    im = ax.imshow(data['values'], cmap="Blues", vmin=0.15, vmax=1)
    _ = ax.set_xticks(np.arange(len(smearing_values)), labels=smearing_values)
    _ = ax.set_yticks(
        np.arange(len(kpoint_distances)), labels=sorted(kpoint_distances, reverse=True)
    )
    _ = ax.set_xlabel("Smearing value (Ry)", fontsize=10)
    _ = ax.set_ylabel("K-point distance (Å$^{-1}$)", fontsize=10)
    _ = place_annotations(data, ax, kpoint_distances, smearing_values)
    _ = write_texts(ax, smearing_values, property_key)
    syst_thr_slider, kpts_thr_slider = place_sliders(
        fig, property_key, slider_lims, default_systematic_error_thr, default_kpts_error_thr)

    callback = partial(
        update_on_slider_change, fig=fig, ax=ax, im=im, all_data=all_data,
        syst_thr_slider=syst_thr_slider, kpts_thr_slider=kpts_thr_slider, 
        available_systematic_thresholds=available_systematic_thresholds,
        available_kpt_thresholds=available_kpt_thresholds,
        kpoint_distances=kpoint_distances, smearing_values=smearing_values,
        property_key=property_key
        ) 

    syst_thr_slider.on_changed(callback)
    kpts_thr_slider.on_changed(callback)
    return syst_thr_slider, kpts_thr_slider


In [5]:
fig1 = create_figure()
fig2 = create_figure()
# Apparently it's important to store them in variables to avoid issues with them going out of scope?
slider1_1, slider1_2 = plot_convergence_heatmap("force_on_disp_atom", fig1)
slider2_1, slider2_2 = plot_convergence_heatmap("energy_per_atom", fig2)

layout = ipw.VBox([fig1.canvas, fig2.canvas])
display(layout)